# Cleaning house listings for sale

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')
#!pip install pydotplus

from IPython.display import Image, display #for tree plot 
import pydotplus 
from scipy import misc

import plotly.express as px

import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)


## Import the data

In [2]:
df=pd.read_csv('RealEstateNewYork.csv',sep=',',low_memory=False)

In [3]:
df.shape

(8652, 15)

In [4]:
df_clean=df.copy()

## Acquaintance with the data

In [5]:
df_clean.describe(include='all')

,price,beds,baths,garage,stories,house_type,lot_sqft,sqft,year_built,address,state,city,lat,lon,county
count,8.652000e+03,8652.000000,8652.000000,4738.000000,6799.000000,8652,7210.000000,5150.000000,8652.000000,8652,8652,8652,8446.000000,8446.000000,8652
unique,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,43,1,39,NaN,NaN,13
top,NaN,NaN,NaN,NaN,NaN,single_family,NaN,NaN,NaN,87-40 Woodhaven Blvd,NY,New York,NaN,NaN,Suffolk
freq,NaN,NaN,NaN,NaN,NaN,5562,NaN,NaN,NaN,206,8652,618,NaN,NaN,2060
mean,1.359977e+06,3.833102,2.761674,1.738919,6.635682,NaN,26553.953398,2016.984466,1957.470527,NaN,NaN,NaN,41.014540,-73.869649,NaN
std,1.833997e+06,1.572397,1.359476,0.605729,14.358759,NaN,42065.077568,753.165533,40.538009,NaN,NaN,NaN,0.563367,1.006523,NaN
min,9.991300e+04,1.000000,1.000000,1.000000,1.000000,NaN,1600.000000,700.000000,1860.000000,NaN,NaN,NaN,40.631451,-78.391600,NaN
25%,5.499000e+05,3.000000,2.000000,1.000000,2.000000,NaN,4000.000000,1440.000000,1930.000000,NaN,NaN,NaN,40.728737,-73.901971,NaN
50%,7.895000e+05,4.000000,3.000000,2.000000,2.000000,NaN,8640.000000,1840.000000,1955.000000,NaN,NaN,NaN,40.776836,-73.753398,NaN
75%,1.199000e+06,5.000000,3.000000,2.000000,3.000000,NaN,23958.000000,2556.000000,1986.000000,NaN,NaN,NaN,40.955878,-73.468086,NaN


In [6]:
df_clean.isnull().sum().sum()

11123

In [7]:
df_clean.isnull().sum()

price            0
beds             0
baths            0
garage        3914
stories       1853
house_type       0
lot_sqft      1442
sqft          3502
year_built       0
address          0
state            0
city             0
lat            206
lon            206
county           0
dtype: int64

## Deal with missing data

### 1. Delete missing values:

In [8]:
df_clean = df_clean.dropna(subset=['city','county']).reset_index(drop=True)
df_clean

,price,beds,baths,garage,stories,house_type,lot_sqft,sqft,year_built,address,state,city,lat,lon,county
0,585000,5,4,2.0,NaN,single_family,135036.0,2560.0,1979,18 Arbor Rd,NY,Campbell Hall,41.449837,-74.231302,Orange
1,735000,3,2,NaN,2.0,single_family,NaN,2556.0,2022,10 Kolb Rd,NY,Boiceville,42.000346,-74.284836,Ulster
2,139900,2,1,2.0,1.0,single_family,180338.0,1210.0,1960,1524 Hardy Corners Rd,NY,Franklinville,42.364357,-78.391600,Cattaraugus
3,540000,2,2,NaN,NaN,single_family,69696.0,1320.0,1980,50 Van Deusen Rd,NY,Hudson,42.214365,-73.752362,Columbia
4,369000,5,3,3.0,2.0,multi_family,14810.0,2751.0,1860,6 Hudson St,NY,Port Jervis,41.379343,-74.690333,Orange
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8647,1200000,5,2,NaN,2.0,multi_family,2000.0,NaN,1900,72-27 66th Pl,NY,Glendale,40.700590,-73.886594,Queens
8648,518000,1,1,1.0,1.0,condos,NaN,NaN,2005,40-53 62nd St Unit 3F,NY,Woodside,40.744014,-73.901971,Queens
8649,7975000,5,6,NaN,52.0,condos,NaN,3429.0,2021,200 Amsterdam Ave Unit 3A,NY,New York,40.776836,-73.983100,New York
8650,7360000,3,4,NaN,52.0,condos,NaN,2221.0,2021,200 Amsterdam Ave Unit 25B,NY,New York,40.776836,-73.983100,New York


In [9]:
df_clean.isnull().sum()

price            0
beds             0
baths            0
garage        3914
stories       1853
house_type       0
lot_sqft      1442
sqft          3502
year_built       0
address          0
state            0
city             0
lat            206
lon            206
county           0
dtype: int64

### 2. Fill NaNs :

In [10]:
df_clean.year_built = df_clean.year_built.fillna(df_clean.year_built.median())
df_clean.beds = df_clean.beds.fillna(df_clean.beds.median())
df_clean.baths = df_clean.baths.fillna(df_clean.baths.median())
df_clean.garage =df_clean.garage.fillna(0, inplace = False)
df_clean.stories = df_clean.stories.fillna(df_clean.stories.median())
df_clean.sqft = df_clean.sqft.fillna(df_clean.sqft.median())
df_clean.lot_sqft = df_clean.lot_sqft.fillna(df_clean.lot_sqft.median())
df_clean.address = df_clean.address.fillna('Not Specified')

In [11]:
df_clean.isnull().sum()

price           0
beds            0
baths           0
garage          0
stories         0
house_type      0
lot_sqft        0
sqft            0
year_built      0
address         0
state           0
city            0
lat           206
lon           206
county          0
dtype: int64

## Drop the unnecessary columns

In [12]:
df_clean = df_clean.drop(columns=(['lon', 'lat']))

## Drop the unnecessary rows

In [13]:
df_clean = df_clean[df_clean["house_type"].str.contains("mobile") == False]
df_clean = df_clean[df_clean["house_type"].str.contains("condo_townhome_rowhome_coop") == False]
df_clean = df_clean[df_clean["house_type"].str.contains("condop") == False]
df_clean = df_clean[df_clean["house_type"].str.contains("farm") == False]

## Remove duplicates

In [14]:
df_clean= df_clean.drop_duplicates().reset_index(drop=True)
df_clean

,price,beds,baths,garage,stories,house_type,lot_sqft,sqft,year_built,address,state,city,county
0,585000,5,4,2.0,2.0,single_family,135036.0,2560.0,1979,18 Arbor Rd,NY,Campbell Hall,Orange
1,735000,3,2,0.0,2.0,single_family,8640.0,2556.0,2022,10 Kolb Rd,NY,Boiceville,Ulster
2,139900,2,1,2.0,1.0,single_family,180338.0,1210.0,1960,1524 Hardy Corners Rd,NY,Franklinville,Cattaraugus
3,540000,2,2,0.0,2.0,single_family,69696.0,1320.0,1980,50 Van Deusen Rd,NY,Hudson,Columbia
4,369000,5,3,3.0,2.0,multi_family,14810.0,2751.0,1860,6 Hudson St,NY,Port Jervis,Orange
5,549900,7,3,0.0,2.0,single_family,4792.0,2990.0,1880,206 Main St,NY,Catskill,Greene
6,339900,3,3,2.0,2.0,single_family,17860.0,2026.0,2004,888 Lothario Cir,NY,Webster,Monroe
7,539000,3,2,2.0,2.0,single_family,13068.0,1700.0,1976,285 Sunrise Dr,NY,Sayville,Suffolk
8,949000,5,4,2.0,2.0,single_family,23958.0,4000.0,2005,114 Alden Dr,NY,Port Jefferson,Suffolk
9,499000,4,1,0.0,3.0,single_family,7000.0,1498.0,1960,5 Bergen Dr,NY,Deer Park,Suffolk


## Changing data types

In [15]:
df_clean['year_built']=df_clean['year_built'].astype(np.int64)

df_clean['beds']=df_clean['beds'].astype(np.float64)

df_clean['baths']=df_clean['baths'].astype(np.float64)

df_clean['stories']=df_clean['stories'].astype(np.int64)

df_clean['lot_sqft']=df_clean['lot_sqft'].astype(np.int64)

df_clean['sqft']=df_clean['sqft'].astype(np.int64)

df_clean['garage']=df_clean['garage'].astype(np.int64)

In [16]:
df_clean.dtypes

price           int64
beds          float64
baths         float64
garage          int64
stories         int64
house_type     object
lot_sqft        int64
sqft            int64
year_built      int64
address        object
state          object
city           object
county         object
dtype: object

## Using pandas describe() to find outliers


In [17]:
df_clean.describe(include='all')

,price,beds,baths,garage,stories,house_type,lot_sqft,sqft,year_built,address,state,city,county
count,4.200000e+01,42.000000,42.000000,42.000000,42.000000,42,42.000000,42.000000,42.000000,42,42,42,42
unique,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,42,1,38,13
top,NaN,NaN,NaN,NaN,NaN,single_family,NaN,NaN,NaN,18 Arbor Rd,NY,New York,Suffolk
freq,NaN,NaN,NaN,NaN,NaN,28,NaN,NaN,NaN,1,42,3,9
mean,1.374228e+06,3.857143,2.785714,0.976190,5.666667,NaN,23553.333333,1936.190476,1955.833333,NaN,NaN,NaN,NaN
std,1.848492e+06,1.570636,1.353148,0.975007,13.017187,NaN,39476.054694,599.330124,40.703519,NaN,NaN,NaN,NaN
min,1.399000e+05,1.000000,1.000000,0.000000,1.000000,NaN,1600.000000,700.000000,1860.000000,NaN,NaN,NaN,NaN
25%,5.737500e+05,3.000000,2.000000,0.000000,2.000000,NaN,4844.000000,1677.500000,1930.000000,NaN,NaN,NaN,NaN
50%,7.895000e+05,4.000000,3.000000,1.000000,2.000000,NaN,8640.000000,1840.000000,1954.000000,NaN,NaN,NaN,NaN
75%,1.199000e+06,5.000000,3.000000,2.000000,2.750000,NaN,15464.000000,1994.500000,1979.750000,NaN,NaN,NaN,NaN


## Detecting & handling outliers

### 1. Price outliers : 

In [18]:
df_clean['price'].describe()

count    4.200000e+01
mean     1.374228e+06
std      1.848492e+06
min      1.399000e+05
25%      5.737500e+05
50%      7.895000e+05
75%      1.199000e+06
max      7.975000e+06
Name: price, dtype: float64

In [19]:
z_score = (df_clean['price'] - df_clean['price'].mean())/df_clean['price'].std()

In [20]:
price_outliers = abs(z_score)>3
sum(price_outliers)

2

In [21]:
min(df_clean.price[price_outliers])

7360000

In [22]:
max(df_clean.price[price_outliers])

7975000

In [23]:
df_clean[(df_clean['price'] < 100000) | (df_clean['price'] > 6890000)]

,price,beds,baths,garage,stories,house_type,lot_sqft,sqft,year_built,address,state,city,county
38,7975000,5.0,6.0,0,52,condos,8640,3429,2021,200 Amsterdam Ave Unit 3A,NY,New York,New York
39,7360000,3.0,4.0,0,52,condos,8640,2221,2021,200 Amsterdam Ave Unit 25B,NY,New York,New York


### 1.1 Handling price outliers :

In [24]:
df_clean['price'] = np.where((df_clean.price<100000),np.nan,df_clean.price)
df_clean['price'] = np.where((df_clean.price>6890000),np.nan,df_clean.price)
df_clean.isnull().sum()

price         2
beds          0
baths         0
garage        0
stories       0
house_type    0
lot_sqft      0
sqft          0
year_built    0
address       0
state         0
city          0
county        0
dtype: int64

In [25]:
df_clean.price = df_clean.price.fillna(df_clean.price.median())
df_clean.isnull().sum()

price         0
beds          0
baths         0
garage        0
stories       0
house_type    0
lot_sqft      0
sqft          0
year_built    0
address       0
state         0
city          0
county        0
dtype: int64

### 2. Bed outliers : 

In [26]:
df_clean['beds'].describe()

count    42.000000
mean      3.857143
std       1.570636
min       1.000000
25%       3.000000
50%       4.000000
75%       5.000000
max       8.000000
Name: beds, dtype: float64

In [27]:
z_score = (df_clean['beds'] - df_clean['beds'].mean())/df_clean['beds'].std()

In [28]:
beds_outliers = abs(z_score)>3
sum(beds_outliers)

0

In [29]:
min(df_clean.beds[beds_outliers])

ValueError: min() arg is an empty sequence

In [ ]:
max(df_clean.beds[beds_outliers])

In [ ]:
df_clean[df_clean['beds']>=9].reset_index(drop=True)

### 2.1 Handling bed outliers :

In [ ]:
df_clean['beds'] = np.where((df_clean.beds>=9),np.nan,df_clean.beds)
df_clean.isnull().sum()


In [ ]:
df_clean=df_clean.dropna(subset=['beds'])
df_clean.isnull().sum()

### 3. Bath outliers : 

In [ ]:
df_clean['baths'].describe()

In [ ]:
z_score = (df_clean['baths'] - df_clean['baths'].mean())/df_clean['baths'].std()

In [ ]:
baths_outliers = abs(z_score)>3
sum(baths_outliers)

In [ ]:
min(df_clean.baths[baths_outliers])

In [ ]:
max(df_clean.baths[baths_outliers])


In [ ]:
df_clean[df_clean['baths']>=7].reset_index(drop=True)

### 3.1 Handling bath outliers :

In [ ]:
df_clean['baths'] = np.where((df_clean.baths>=7),np.nan,df_clean.baths)
df_clean.isnull().sum()

In [ ]:
df_clean=df_clean.dropna(subset=['baths'])
df_clean.isnull().sum()

### 4. Garage outliers : 

In [ ]:
df_clean['garage'].describe()

In [ ]:
z_score = (df_clean['garage'] - df_clean['garage'].mean())/df_clean['garage'].std()

In [ ]:
garage_outliers = abs(z_score)>3
sum(garage_outliers)

In [ ]:
min(df_clean.garage[garage_outliers])

In [ ]:
max(df_clean.garage[garage_outliers])

In [ ]:
df_clean[df_clean['garage']>=7].reset_index(drop=True)

### 4.1 Handling garage outliers :

In [ ]:
df_clean['garage'] = np.where((df_clean.garage>=7),np.nan,df_clean.garage)
df_clean.isnull().sum()

In [ ]:
df_clean.garage = df_clean.garage.fillna(df_clean.garage.median())
df_clean.isnull().sum()

### 5. Sqft outliers : 

In [ ]:
df_clean['sqft'].describe()

In [ ]:
z_score = (df_clean['sqft'] - df_clean['sqft'].mean())/df_clean['sqft'].std()

In [ ]:
sqft_outliers = abs(z_score)>3
sum(sqft_outliers)

In [ ]:
min(df_clean.sqft[sqft_outliers])

In [ ]:
max(df_clean.sqft[sqft_outliers])

In [ ]:
df_clean[df_clean['sqft']<320].reset_index(drop=True)

In [ ]:
df_clean[df_clean['sqft']>=36590].reset_index(drop=True)

### 5.1 Handling sqft outliers :

In [ ]:
df_clean['sqft'] = np.where((df_clean.sqft>=36590),np.nan,df_clean.sqft)
df_clean['sqft'] = np.where((df_clean.sqft<=300),np.nan,df_clean.sqft)
df_clean.isnull().sum()

In [ ]:
df_clean.sqft = df_clean.sqft.fillna(df_clean.sqft.median())
df_clean.isnull().sum()

### 6. Lot Sqft outliers :

###          6.1 Removing values of "Lot Sqft" that are smaller then "Sqft" :

In [ ]:
df_clean

In [ ]:
count=0

for i, j in df_clean.iterrows():
    if((df_clean['lot_sqft'][i]) < (df_clean['sqft'][i])):
        count+=1
        df_clean.drop([i], axis=0, inplace=True)

print(count)
print()

df_clean = df_clean.reset_index(drop=True)
df_clean

### 6.2 The Outliers : 

In [ ]:
df_clean['lot_sqft'].describe()

In [ ]:
z_score = (df_clean['lot_sqft'] - df_clean['lot_sqft'].mean())/df_clean['lot_sqft'].std()

In [ ]:
lot_sqft_outliers = abs(z_score)>3
sum(lot_sqft_outliers)

In [ ]:
min(df_clean.lot_sqft[lot_sqft_outliers])

In [ ]:
max(df_clean.lot_sqft[lot_sqft_outliers])

In [ ]:
df_clean[df_clean['lot_sqft']>=18360540].reset_index(drop=True)

### 6.2.1 Handling Lot Sqft outliers 

In [ ]:
df_clean['lot_sqft'] = np.where((df_clean.lot_sqft>=18360540),np.nan,df_clean.lot_sqft)
df_clean.isnull().sum()

In [ ]:
df_clean=df_clean.dropna(subset=['lot_sqft'])
df_clean.isnull().sum()


In [ ]:
df_clean

### 7. Stories outliers :

In [ ]:
df_clean['stories'].describe()

In [ ]:
z_score = (df_clean['stories'] - df_clean['stories'].mean())/df_clean['stories'].std()

In [ ]:
stories_outliers = abs(z_score)>3
sum(stories_outliers)

In [ ]:
min(df_clean.stories[stories_outliers])

In [ ]:
max(df_clean.stories[stories_outliers])

In [ ]:
df_clean[df_clean['stories']>=20].reset_index(drop=True)

### 7.1 Handling sqft outliers :

In [ ]:
df_clean['stories'] = np.where((df_clean.stories>=20),np.nan,df_clean.stories)
df_clean.isnull().sum()

In [ ]:
df_clean=df_clean.dropna(subset=['stories'])
df_clean.isnull().sum()

In [ ]:
df_clean = df_clean.reset_index(drop=True)
df_clean

## Feature Engineering

In [ ]:
df_clean['price_per_sqft'] = df_clean.apply(lambda row: row.price / row.sqft, axis=1)
df_clean


### price_per_sqft outliers :

In [ ]:
df_clean['price_per_sqft'].describe()

In [ ]:
z_score = (df_clean['price_per_sqft'] - df_clean['price_per_sqft'].mean())/df_clean['price_per_sqft'].std()

In [ ]:
price_per_sqft_outliers = abs(z_score)>3
sum(price_per_sqft_outliers)

In [ ]:
min(df_clean.price_per_sqft[price_per_sqft_outliers])

In [ ]:
max(df_clean.price_per_sqft[price_per_sqft_outliers])

In [ ]:
df_clean[df_clean['price_per_sqft']<50].reset_index(drop=True)

In [ ]:
df_clean[df_clean['price_per_sqft']>2500].reset_index(drop=True)

### Handling sqft outliers :

In [ ]:
df_clean['price_per_sqft'] = np.where((df_clean.price_per_sqft<50),np.nan,df_clean.price_per_sqft)


In [ ]:
df_clean['price_per_sqft'] = np.where((df_clean.price_per_sqft>2500),np.nan,df_clean.price_per_sqft)
df_clean.isnull().sum()

In [ ]:
df_clean=df_clean.dropna(subset=['price_per_sqft'])
df_clean.isnull().sum()

In [ ]:
df_clean = df_clean.reset_index(drop=True)
df_clean

## Counting categorical features

In [ ]:
df_clean['house_type'].value_counts()

In [ ]:
df_clean["house_type"].replace({"apartment": "condos"}, inplace=True)

In [ ]:
df_clean['house_type'].value_counts()

In [ ]:
df_clean['city'].value_counts()

In [ ]:
df_clean["city"].replace({"New York": "New York City"}, inplace=True)

In [ ]:
df_clean['city'].value_counts()

In [ ]:
df_clean.to_csv('RealEstateNewYork_Clean2.csv')